# Análisis de Crímenes 2024

Este notebook contiene el análisis exploratorio de datos sobre crímenes reportados en 2024.

## Importación de Librerías

Importamos las bibliotecas necesarias para el análisis de datos:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_csv('lib/Crimes_-_2024_20251030.csv')

# Mostrar las primeras filas y la información básica del dataset
print("Primeras 5 filas del dataset:")
display(df.head())

